# LSTM 

In [1]:
%load_ext autoreload
%autoreload 2
# В юпитер ноутбуке если вносишь изменения в .py откуда делаются импорты функций и классов, ноутбук прогоняет старые версии, не замечая изменений.
# Эти импорты это исправляют

In [5]:
import pickle                  # Для загрузки словаря
import numpy as np             # математические операции
import pandas as pd            # работа с датафреймами
from string import punctuation # для удаления пунктуации

from nltk.corpus import stopwords # для удаления стопслов
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec # нейронная сеть для эмбэддинга на принципе контекста

import torch                                            # фреймворк для машинного обучения и нейросетей
from torch.utils.data import DataLoader, TensorDataset  # DataLoader для загрузки данных в модель батчами
import torch.nn as nn                                   # TensorDataset для создания датасета из Тензора
import torchutils as tu                                 # get_model_summary()
from torchmetrics.classification import (  
    BinaryAccuracy, 
    AUROC
    )
from aux.rnn_preprocessing import (                         # Импортируем из собственного файла функции для препросессинга
                                data_preprocessing, 
                                preprocess_single_string,
                                predict_sentence
                                )

from gensim.models import KeyedVectors # Чтобы прочитать сохраненные векторы

Загружаю вектора сохраненные с коллаба

In [7]:
wv = KeyedVectors.load("aux/wv.wordvectors", mmap='r')

with open('aux/vocab_to_int.txt', 'rb') as f:
    vocab_to_int = pickle.load(f)

In [8]:
# Создаем слой эмбеддинга
# -  wv[word]  обращается к вектору слова  word  в модели Word2Vec 
# и присваивает его переменной  embedding_vector . 

VOCAB_SIZE = len(vocab_to_int) + 1   
EMBEDDING_DIM = 32

embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in vocab_to_int.items():
    try:
        embedding_vector = wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError as e:
        # print(f'word: {word}')
        pass
        

embedding_layer = torch.nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix)) # from_pretrained потому что мы его сами тренировали
print(f'EMBEDDING MATRIX SHAPE = number of words x EMEDDING_DIM: {embedding_matrix.shape}')

EMBEDDING MATRIX SHAPE = number of words x EMEDDING_DIM: (196906, 32)


Задали embedding_layer!

In [9]:
DEVICE = 'cuda' #cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim: int, hidden_size:int = 32) -> None:
        super().__init__()

        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.embedding = embedding_layer
        
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            batch_first=True
        )
        self.clf = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        _, (h_n, _) = self.lstm(embeddings)
        out = self.clf(h_n.squeeze())
        return out

# EMBEDDING_DIM уже указал при создании embedding_matrix
HIDDEN_DIM = 32
SEQ_LEN = 200      
model = LSTMClassifier(embedding_dim=EMBEDDING_DIM, hidden_size=HIDDEN_DIM).to(DEVICE)


RuntimeError: No CUDA GPUs are available

Предсказываем

In [ ]:
pred = predict_sentence('Красивая, чистая больница, доктора добрые, в восторге', model)